In [ ]:
import os
import sys
import datetime
from pathlib import Path
import logging
from main import SigmoidFitter, MeltomeAtlasHandler, setup_logging

import json
import numpy as np
import pandas as pd
import seaborn as sns
# for logfile in [f for f in os.listdir() if '.log' in f]:
#     print(logfile)
    # os.remove(logfile)
now = datetime.datetime.now()
timestamp_str = now.strftime("%Y-%m-%d_%H-%M")
setup_logging(f'./testing_{timestamp_str}.log', logging.DEBUG)

2025-12-10 17:29:16 - root - INFO - Logging to file: ./2025-12-10_17-29_testing.log


In [20]:
# SCRIPT_DIR = Path(__file__).resolve().parent
CWD = Path(os.getcwd()).resolve()
PROJECT_ROOT = CWD.parent.parent
print(PROJECT_ROOT)
file_path = PROJECT_ROOT / "Data" / "FLIP" / "splits" / "meltome" / "full_dataset.json"
output_path = f'./results_meltome_{timestamp_str}'
print(file_path)

C:\Users\alexa\Documents\PROHITS
C:\Users\alexa\Documents\PROHITS\Data\FLIP\splits\meltome\full_dataset.json


In [21]:
# Sample subset of Meltome
data_handler = MeltomeAtlasHandler(file_path, output_path, logging.DEBUG)
print(data_handler.data.shape)
df = data_handler.select_subset(100)
chunk0 = data_handler.data.iloc[:25]
data_handler.data = df 
print(data_handler.data.shape)

2025-12-10 17:29:16 - MeltomeAtlasHandler - INFO - Meltome Handler initialization
2025-12-10 17:29:16 - MeltomeAtlasHandler - DEBUG - DEBUG Mode
2025-12-10 17:29:19 - MeltomeAtlasHandler - INFO - Meltome data loaded from C:\Users\alexa\Documents\PROHITS\Data\FLIP\splits\meltome\full_dataset.json
2025-12-10 17:29:19 - MeltomeAtlasHandler - INFO - Meltome header ['proteinId', 'uniprotAccession', 'runName', 'meltingPoint', 'meltingBehaviour', 'sequence', 'quantNormMeltingPoint']
2025-12-10 17:29:19 - MeltomeAtlasHandler - INFO - Output directory: c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\results_meltome_2025-12-10_17-29
(221203, 7)
(2500, 7)


In [22]:
n = np.arange(20)
s = np.array_split(n, 10)
print(data_handler.data.shape[0], len(s), np.sum([len(i) for i in s]))

2500 10 20


In [23]:
test_data = pd.DataFrame(data_handler.data.iloc[0].meltingBehaviour)

In [24]:
test_data

,temperature,channel,fold_change
0,25.0,TMT126,1.000000
1,36.0,TMT127H,1.020938
2,30.0,TMT127L,0.988506
3,45.0,TMT128H,0.872294
4,39.0,TMT128L,0.949953
5,56.0,TMT129H,0.318029
6,50.0,TMT129L,0.679647
7,66.9,TMT130H,0.832007
8,59.6,TMT130L,0.627966
9,70.4,TMT131L,1.078205


In [25]:
t = SigmoidFitter()
print(t.get_statistics())
r = t.fit_curve(test_data.temperature.to_numpy(), test_data.fold_change.to_numpy())
print(t.get_statistics())

{'rmse': nan, 'r_squared': nan}
{'rmse': np.float64(0.1774227960743552), 'r_squared': np.float64(0.36193825549188574)}


In [26]:
x = t.get_melting_temp()
print(x)

55.07


c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: divide by zero encountered in divide
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


In [27]:
results0 = data_handler.process_chunk(chunk0)

2025-12-10 17:29:20 - MeltomeAtlasHandler - INFO - Chunk progress : 0 / 25, pid : A0A023PXQ4, specie : Saccharomyces cerevisiae lysate
2025-12-10 17:29:20 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:29:20 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9918, RMSE = 0.0280
2025-12-10 17:29:20 - SigmoidFitter - DEBUG - Fitted parameters: pl = -0.387047, a = 396.281525, b = 6.904978
2025-12-10 17:29:20 - MeltomeAtlasHandler - INFO - Chunk progress : 1 / 25, pid : A0A023T4K3, specie : Caenorhabditis elegans lysate
2025-12-10 17:29:20 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:29:20 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9867, RMSE = 0.0453
2025-12-10 17:29:20 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.052359, a = 719.635395, b = 19.066874
2025-12-10 17:29:20 - MeltomeAtlasHandler - INFO - Chunk progress : 2 / 25, pid : A0A023T778, specie : Mus musculus BMDC lysate
2025-12-10 17:29:20 - SigmoidFitter - DEBU

c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: overflow encountered in exp
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


In [28]:
results0_df = pd.DataFrame(results0)

In [29]:
data_handler.data

,proteinId,uniprotAccession,runName,meltingPoint,meltingBehaviour,sequence,quantNormMeltingPoint
37797,Q9ZUL3_IDD5,Q9ZUL3,Arabidopsis thaliana seedling lysate,NaN,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MAASSSSAASFFGVRQDDQSHLLPPNSSAAAPPPPPPHHQAPLPPL...,NaN
17375,P93736_TWN2,P93736,Arabidopsis thaliana seedling lysate,41.318207,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MSLLFLRRAKPLFVSCCSATHSRSSFLSPTLTNQLVRSFHGSRTMS...,NaN
34236,Q9FJW4_FRO1,Q9FJW4,Arabidopsis thaliana seedling lysate,NaN,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MALCATTQRTIRIAATLRRVARPFATDAVVESDYKRGEIGKVSGIP...,NaN
34859,Q9LP45_RPN6,Q9LP45,Arabidopsis thaliana seedling lysate,43.522499,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MVSYRATTETISLALEANSSEAITILYQVLEDPSSSPEAIRIKEQA...,NaN
2869,C0LGQ9_At4g20940,C0LGQ9,Arabidopsis thaliana seedling lysate,42.575046,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MNLSRILLLSMFFLSAMGQLPSQDIMALLEFKKGIKHDPTGFVLNS...,NaN
...,...,...,...,...,...,...,...
64783,CAND1,Q86VP6,pTcells,55.656613,"[{'temperature': 37.0, 'fold_change': 0.999985...",MASASYHISNLLEKMTSSDKDFRFMATNDLMTELQKDSIKLDDDSE...,56.869709
81901,DECR1,Q16698,pTcells,57.770579,"[{'temperature': 37.0, 'fold_change': 0.999985...",MKLPARVFFTLGSRLPCGLAPRRFFSYGTKILYQNTEALQSKFFSP...,58.688807
221084,ZYX,Q15942,pTcells,58.301234,"[{'temperature': 37.0, 'fold_change': 0.999985...",MAAPRPSPAISVSVSAPAFYAPQKKFGPVVAPKPKVNPFRPGDSEP...,59.699714
169838,RASSF2,P50749,pTcells,47.268902,"[{'temperature': 37.0, 'fold_change': 0.999985...",MDYSHQTSLVPCGQDKYISKNELLLHLKTYNLYYEGQNLQLRHREE...,46.627860


In [30]:
results0_df

,pid,runName,pl,a,b,rmse,r_squared,tm_pred,tm_flip
0,A0A023PXQ4,Saccharomyces cerevisiae lysate,-0.387047,396.281525,6.904978,0.0280,0.9918,52.99,52.403034
1,A0A023T4K3,Caenorhabditis elegans lysate,0.052359,719.635395,19.066874,0.0453,0.9867,37.96,37.962947
2,A0A023T778,Mus musculus BMDC lysate,0.106919,1749.919673,32.061066,0.0564,0.9812,54.99,54.425342
3,A0A061ACF5,Caenorhabditis elegans lysate,0.278508,295.604929,6.790961,0.0246,0.9868,49.46,49.459216
4,A0A061ACH4,Caenorhabditis elegans lysate,0.023551,724.433779,17.056474,0.0390,0.9914,42.59,42.593131
5,A0A061ACH8,Caenorhabditis elegans lysate,0.052800,559.097425,14.824896,0.0189,0.9977,38.00,37.999478
6,A0A061ACH9,Caenorhabditis elegans lysate,0.022312,602.364195,14.213085,0.0412,0.9878,42.52,42.517527
7,A0A061ACI3,Caenorhabditis elegans lysate,0.237867,660.532837,15.694724,0.0486,0.9727,43.89,43.892211
8,A0A061ACL3,Caenorhabditis elegans lysate,0.081819,934.223132,22.436277,0.0397,0.9909,41.97,41.973245
9,A0A061ACL6,Caenorhabditis elegans lysate,-1152.945257,224.780039,-3.236776,0.0587,0.9484,49.87,49.044775


In [ ]:
results = data_handler.process(num_chunks=10)

2025-12-10 17:29:27 - MeltomeAtlasHandler - INFO - Starting curve fitting process
2025-12-10 17:29:27 - MeltomeAtlasHandler - DEBUG - Data split into 10 chunks
2025-12-10 17:29:27 - MeltomeAtlasHandler - INFO - Processing chunk 1 / 10 (size : 250)
2025-12-10 17:29:27 - MeltomeAtlasHandler - INFO - Chunk progress : 0 / 250, pid : Q9ZUL3, specie : Arabidopsis thaliana seedling lysate
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.3619, RMSE = 0.1774
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.707171, a = 8974.218952, b = 199.170123
2025-12-10 17:29:27 - MeltomeAtlasHandler - INFO - Chunk progress : 1 / 250, pid : P93736, specie : Arabidopsis thaliana seedling lysate
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9984, RMSE = 0.0158
2025-12-10 17:29:27 - Si

c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: divide by zero encountered in divide
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9784, RMSE = 0.0594
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Fitted parameters: pl = -0.031404, a = 520.818219, b = 10.953354
2025-12-10 17:29:27 - MeltomeAtlasHandler - INFO - Chunk progress : 8 / 250, pid : O64733, specie : Arabidopsis thaliana seedling lysate
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9255, RMSE = 0.0980
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.217659, a = 527.927364, b = 11.850246
2025-12-10 17:29:27 - MeltomeAtlasHandler - INFO - Chunk progress : 9 / 250, pid : Q9LQK7, specie : Arabidopsis thaliana seedling lysate
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9765, RMSE = 0.0529
2025-12-10 17:29:27 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.

c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: overflow encountered in exp
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


2025-12-10 17:29:50 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9829, RMSE = 0.0341
2025-12-10 17:29:50 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.335451, a = 500.248736, b = 12.458715
2025-12-10 17:29:50 - MeltomeAtlasHandler - INFO - Chunk progress : 248 / 250, pid : G5EEG7, specie : Caenorhabditis elegans lysate
2025-12-10 17:29:50 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:29:50 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9992, RMSE = 0.0106
2025-12-10 17:29:50 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.076907, a = 549.056253, b = 14.692952
2025-12-10 17:29:50 - MeltomeAtlasHandler - INFO - Chunk progress : 249 / 250, pid : O01530, specie : Caenorhabditis elegans lysate
2025-12-10 17:29:50 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:29:50 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.8581, RMSE = 0.0868
2025-12-10 17:29:50 - SigmoidFitter - DEBUG - Fitted parameters: pl = -2474.651184,

In [ ]:
print(len(data_handler.data), len(results))

2500 2500


In [ ]:
pid, runName = results.loc[0][['pid', 'runName']]
print(pid, runName)
data_handler.data[(data_handler.data.uniprotAccession == pid) & (data_handler.data.runName == runName)]


O64527 Arabidopsis thaliana seedling lysate


,proteinId,uniprotAccession,runName,meltingPoint,meltingBehaviour,sequence,quantNormMeltingPoint
7330,O64527_At1g79260,O64527,Arabidopsis thaliana seedling lysate,50.994332,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MNQLQQLQNPGESPPVHPFVAPLSYLLGTWRGQGEGEYPTIPSFRY...,NaN


In [ ]:
results[results.notna().all(axis=1)]

,pid,runName,pl,a,b,rmse,r_squared,tm_pred,tm_flip
0,O64527,Arabidopsis thaliana seedling lysate,0.480290,9666.344264,192.747918,0.1371,0.8107,51.01,50.994332
1,Q9XGN1,Arabidopsis thaliana seedling lysate,-0.015506,357.205893,7.801045,0.0343,0.9914,45.61,45.543917
2,Q96250,Arabidopsis thaliana seedling lysate,0.066677,325.638176,7.904054,0.0411,0.9862,41.96,41.958675
3,O23461,Arabidopsis thaliana seedling lysate,-0.026901,280.736422,6.767107,0.0366,0.9891,41.17,41.068258
5,Q949Y3,Arabidopsis thaliana seedling lysate,0.120878,748.430882,15.652913,0.0385,0.9894,48.67,48.674775
...,...,...,...,...,...,...,...,...,...
2494,Q9BW92,pTcells,0.001974,932.844699,18.342615,0.0303,0.9942,50.87,50.867660
2496,O94819,pTcells,0.032668,1536.259578,30.188189,0.0255,0.9964,51.00,51.003582
2497,P28062,pTcells,0.074429,1619.545803,27.285654,0.1159,0.9012,59.71,59.707886
2498,Q6P9B9,pTcells,-0.068523,532.517260,11.535357,0.1309,0.8670,45.66,45.594255


In [ ]:
results_na = results[results.isna().any(axis=1)]
print(results_na.shape)

(451, 9)


In [ ]:
results_na

,pid,runName,pl,a,b,rmse,r_squared,tm_pred,tm_flip
4,Q9C5W7,Arabidopsis thaliana seedling lysate,1.756697,12134.482876,183.006511,0.1844,0.7377,0.00,NaN
13,Q8VYI9,Arabidopsis thaliana seedling lysate,2.249051,9347.596178,139.196358,0.2198,0.7386,0.00,NaN
20,P41127,Arabidopsis thaliana seedling lysate,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,P25855,Arabidopsis thaliana seedling lysate,0.844038,7066.827534,180.860206,0.2445,0.0954,48.64,NaN
24,Q96528,Arabidopsis thaliana seedling lysate,NaN,NaN,NaN,NaN,NaN,NaN,59.488719
...,...,...,...,...,...,...,...,...,...
2321,Q13496,colon_cancer_spheroids,NaN,NaN,NaN,NaN,NaN,NaN,60.978652
2348,Q9NUD5,colon_cancer_spheroids,NaN,NaN,NaN,NaN,NaN,NaN,56.771614
2356,Q8TCF1,colon_cancer_spheroids,NaN,NaN,NaN,NaN,NaN,NaN,53.944857
2451,None,pTcells,0.036313,1580.807508,32.923032,0.0301,0.9947,48.13,48.125463
